# Train a SVM on word embeddings generated by BERTje

This notebook trains an Support Vector Machine on the word embeddings generated by the BERTje model. These embeddings are used as the input features for the SVM which classifies wether tweets are abusive, offensive or not.

The notebook can be ran locally or on a service like google colab.

In [ ]:
# Install dependencies
!pip uninstall accelerate transformers -y
!pip install -U accelerate>=0.21.0
!pip3 install transformers
!pip3 install datasets
!pip3 install pandas
!pip3 install torch
!pip3 install scikit-learn
!pip3 install numpy
!pip3 install nltk emoji==0.6.0

Found existing installation: transformers 4.38.2
Uninstalling transformers-4.38.2:
  Successfully uninstalled transformers-4.38.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 931.2 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-0.6.0-py3-none-any.whl size=49720 sha256=67c15c3065346139bde5ef3ea1925c10c562723f4a5eeaf32cd42303ea98f9ce
  Stored in directory: /root/.cache/pip/wheels/1b/bd/d9/310c33c45a553798a714e27e3b8395d37128425442b8c78e07
Successfully built emoji


In [ ]:
# Load the data
import pandas as pd

dataset = pd.read_csv("train_data_offensive_abusive_taskC.csv")

# Split data 80 / 20
dataset_len = len(dataset)

# Shuffle the DataFrame to ensure randomness
dataset = dataset.sample(frac=1, random_state=42)

# Split the data in train and test
train = dataset.tail(int(dataset_len*0.8))
train_X = train['text'].tolist()
train_y = train['abusive_offensive_not'].tolist()

test = dataset.head(int(dataset_len*0.2))
test_X = test['text'].tolist()
test_y = test['abusive_offensive_not'].tolist()

In [ ]:
import torch
import numpy as np
from tqdm import tqdm

# Get the embeddings from BERTje
model = AutoModel.from_pretrained("GroNLP/bert-base-dutch-cased")

def get_embeddings(data):
  # Split your data into smaller batches
  batch_size = 8
  num_samples = len(data)
  embeddings = []

  # Process data in batches
  for i in tqdm(range(0, num_samples, batch_size)):
      batch_texts = data[i:i+batch_size]
      batch_tokenized = tokenizer(batch_texts, padding=True, truncation=True, return_tensors="pt")

      with torch.no_grad():
          batch_outputs = model(**batch_tokenized)
          batch_embeddings = batch_outputs.last_hidden_state.mean(dim=1).numpy()
          embeddings.append(batch_embeddings)

  # Concatenate embeddings from all batches
  embeddings = np.concatenate(embeddings, axis=0)

  return embeddings


Some weights of BertModel were not initialized from the model checkpoint at GroNLP/bert-base-dutch-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
embeddings_train = get_embeddings(train_X)

100%|██████████| 682/682 [05:37<00:00,  2.02it/s]


In [ ]:
embeddings_test = get_embeddings(test_X)

100%|██████████| 171/171 [01:27<00:00,  1.96it/s]


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pickle

def save_embeddings(path, data):
  # Save the embeddings using pickle
  with open(path, 'wb') as f:
      pickle.dump(data, f)

  print("Embeddings saved successfully.")

save_embeddings("/content/gdrive/MyDrive/Uni-23-24/Machine Learning Project/Final Assignment/models/bertje_svm/bertje_embeddings_train.pkl", embeddings_train)
save_embeddings("/content/gdrive/MyDrive/Uni-23-24/Machine Learning Project/Final Assignment/models/bertje_svm/bertje_embeddings_test.pkl", embeddings_test)

Embeddings saved successfully.
Embeddings saved successfully.


In [ ]:
from sklearn.svm import SVC

# The SVM
svm_classifier = SVC(kernel='linear')
svm_classifier.fit(embeddings_train, train_y)

SVC(kernel='linear')

In [ ]:
from sklearn.metrics import f1_score

predicted = svm_classifier.predict(embeddings_test)

f1score = f1_score(test_y, predicted, average='macro')

print(f1score)

0.5561369727148254


In [ ]:
# Save the sklearn SVM model
import pickle

with open('/content/gdrive/MyDrive/Uni-23-24/Machine Learning Project/Final Assignment/models/bertje_svm/model.pkl', 'wb') as outfile:
  pickle.dump(svm_classifier, outfile)